<a href="https://colab.research.google.com/github/papaya147/predictive-analysis/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://ora.ox.ac.uk/objects/uuid:99d7c092-d865-4a19-b096-cc16440cd001/files/rpr76f381b

--2024-11-06 05:04:24--  https://ora.ox.ac.uk/objects/uuid:99d7c092-d865-4a19-b096-cc16440cd001/files/rpr76f381b
Resolving ora.ox.ac.uk (ora.ox.ac.uk)... 129.67.246.216
Connecting to ora.ox.ac.uk (ora.ox.ac.uk)|129.67.246.216|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘rpr76f381b’

rpr76f381b              [ <=>                ]   6.43G  19.7MB/s    in 6m 31s  

2024-11-06 05:10:56 (16.8 MB/s) - ‘rpr76f381b’ saved [6902652480]



In [2]:
!unzip rpr76f381b -d data

Archive:  rpr76f381b
   creating: data/capture24/
  inflating: data/capture24/P023.csv.gz  
  inflating: data/capture24/P026.csv.gz  
  inflating: data/capture24/P070.csv.gz  
  inflating: data/capture24/P034.csv.gz  
  inflating: data/capture24/P031.csv.gz  
  inflating: data/capture24/P043.csv.gz  
  inflating: data/capture24/P011.csv.gz  
  inflating: data/capture24/P135.csv.gz  
  inflating: data/capture24/P028.csv.gz  
  inflating: data/capture24/P099.csv.gz  
  inflating: data/capture24/P053.csv.gz  
  inflating: data/capture24/P035.csv.gz  
  inflating: data/capture24/P030.csv.gz  
  inflating: data/capture24/P037.csv.gz  
  inflating: data/capture24/P029.csv.gz  
  inflating: data/capture24/P095.csv.gz  
  inflating: data/capture24/P058.csv.gz  
  inflating: data/capture24/P089.csv.gz  
  inflating: data/capture24/P150.csv.gz  
  inflating: data/capture24/metadata.csv  
  inflating: data/capture24/P097.csv.gz  
  inflating: data/capture24/P072.csv.gz  
  inflating: data/capture

In [3]:
import pandas as pd

In [5]:
data = pd.read_csv("data/capture24/P001.csv.gz", compression="gzip")
data

<ipython-input-5-87b1942fbbd3>:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("data/capture24/P001.csv.gz", compression="gzip")


,time,x,y,z,annotation
0,2016-11-13 02:18:00.000000,-0.466690,-0.533341,0.658472,7030 sleeping;MET 0.95
1,2016-11-13 02:18:00.010000,-0.466690,-0.533341,0.658472,7030 sleeping;MET 0.95
2,2016-11-13 02:18:00.020000,-0.466690,-0.533341,0.658472,7030 sleeping;MET 0.95
3,2016-11-13 02:18:00.030000,-0.466690,-0.533341,0.658472,7030 sleeping;MET 0.95
4,2016-11-13 02:18:00.040000,-0.466690,-0.533341,0.658472,7030 sleeping;MET 0.95
...,...,...,...,...,...
10019996,2016-11-14 06:07:59.960000,0.049416,-0.797846,0.565700,7030 sleeping;MET 0.95
10019997,2016-11-14 06:07:59.970000,0.049416,-0.782285,0.565700,7030 sleeping;MET 0.95
10019998,2016-11-14 06:07:59.980000,0.049416,-0.782285,0.565700,7030 sleeping;MET 0.95
10019999,2016-11-14 06:07:59.990000,0.049416,-0.782285,0.565700,7030 sleeping;MET 0.95


In [9]:
annot = pd.read_csv("data/capture24/annotation-label-dictionary.csv")
annot

,annotation,label:WillettsSpecific2018,label:WillettsMET2018,label:DohertySpecific2018,label:Willetts2018,label:Doherty2018,label:Walmsley2020
0,7030 sleeping;MET 0.95,sleep,sleep,sleep,sleep,sleep,sleep
1,occupation;office and administrative support;1...,sitting,sitstand+lowactivity,sedentary-screen,sit-stand,sedentary,sedentary
2,home activity;household chores;preparing meals...,household-chores,sitstand+activity,tasks-moderate,mixed,moderate,light
3,occupation;office and administrative support;1...,sitting,sitstand+lowactivity,sedentary-screen,sit-stand,sedentary,sedentary
4,home activity;miscellaneous;sitting;9060 sitti...,sitting,sitstand+lowactivity,sedentary-non-screen,sit-stand,sedentary,sedentary
...,...,...,...,...,...,...,...
201,transportation;walking;17250 walking as the si...,mixed-activity,walking,walking,mixed,walking,moderate-vigorous
202,transportation;walking;17270 walking as the si...,walking,walking,walking,walking,walking,moderate-vigorous
203,transportation;public transportation;16016 rid...,vehicle,vehicle,vehicle,vehicle,sedentary,sedentary
204,household-chores;sitstand+lowactivity;MET 2.8,household-chores,sitstand+lowactivity,tasks-light,mixed,tasks-light,light


In [11]:
# lets use Walmsley2020
annot.index = annot["annotation"]
annot = annot["label:Walmsley2020"]
annot

KeyError: 'annotation'